# Datenvisualisierung


<div style="text-align: center;">
<img width="30%" src="https://images.unsplash.com/photo-1638898407927-79801f46960b?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D&auto=format&fit=crop&w=2070&q=80" alt="Peek-a-boo! by Bobby Mc Leod on Unsplash" style="min-width: 300px"/>
<div><small><em><a href="https://unsplash.com/photos/XmnfnuqPNMY">Quelle</a></em></small></div>
</div>

## Lernpfad

1. **Warum ist Visualisierung wichtig?**
1. Plotting mit Pandas und Matplotlib
1. Plotting mit Seaborn
1. Anpassen von Matplotlib-Plots
1. Interaktive Visualisierung mit Plotly

## Wofür brauchen wir Visualisierung?


### Am Anfang des Analyseprozesses...

- Daten verstehen
- Muster und Outlier identifizieren
- Datenqualität validieren

Also: möglichst schnell viele Plots erzeugen.

### ...und am Ende

- Daten für andere verständlich machen
- Erkenntnisse in Reports oder Dashboards kommunizieren

Hier brauchen wir wenige, aber polierte Plots.

**Seid euch klar darüber, welches von beidem ihr gerade wollt.**

Reichen nicht die statistischen Werte, um die Daten zu verstehen?

<div style="text-align: center; margin-top: -10px;">
<img width="50%" src="https://raw.githubusercontent.com/stefmolin/data-morph/main/docs/_static/panda-to-star-eased.gif" alt="Data Morph: panda to star" style="min-width: 300px; margin-bottom: -10px;"/>
<div style="margin: auto 26%;"><small><em>Eine Punktwolke, die einen Panda abbildet, kann auch einen Stern abbilden, ohne dass sich dabei die deskriptiven Statistiken hier signifikant verändern. (Quelle: <a href="https://github.com/stefmolin/data-morph">Data Morph</a>)</em></small></div>
</div>

## Lernpfad

1. Warum ist Visualisierung wichtig?
1. **Plotting mit Pandas und Matplotlib**
1. Plotting mit Seaborn
1. Anpassen von Matplotlib-Plots
1. Interaktive Visualisierung mit Plotly

## Plotting mit Pandas und Matplotlib

Jeder DataFrame hat eine `plot()` Method, die als Wrapper um *Matplotlib* eine Reihe von Visualisierungen einfach zugänglich macht. 

Wir benutzen das TSA Dataset, das wir im letzen Abschnitt vorbereitet haben:

In [ ]:
import pandas as pd

tsa_melted_holiday_travel = pd.read_csv(
    '../data/tsa_melted_holiday_travel.csv', 
    parse_dates=True, index_col='date'
)
tsa_melted_holiday_travel.head()

Im Notebook wollen wir, dass das Matplotlib Plotting Backend Plots im SVG-Format erzeugt.

In [ ]:
import matplotlib_inline

matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

### Liniendiagramm

Die `plot()` Methode erzeugt standardmäßig ein Liniendiagramm mit allen numerischen Spalten:

In [ ]:
tsa_2020 = tsa_melted_holiday_travel.loc['2020'].copy().drop(columns=['year'])

tsa_2020['7D mean'] = tsa_2020.rolling('7D').travelers.mean()
tsa_2020['YTD mean'] = tsa_2020.expanding().travelers.mean()

tsa_2020.plot(title='2020 TSA Traveler Throughput', ylabel='travelers', alpha=0.8)

Die `plot()` Methode erzeugt ein `Axes` Objekt, das weiter modifiziert werden kann. 

### Balkendiagramme

Im nächsten Beispiel werden wir monatliche Fluggastzahlen plotten, jeweils für alle Jahre übereinander.

Dazu brauchen wir erst eine Pivot Table:

In [ ]:
plot_data = tsa_melted_holiday_travel\
    .assign(month=tsa_melted_holiday_travel.index.month)\
    .pivot_table(index='month', columns='year', values='travelers', aggfunc='sum')
plot_data.head()

Für ein Balkendiagramm nutzen wir `kind='bar'` im Aufruf von `plot()`. 

Das `Axes` Objekt passen wir noch etwas an.

In [ ]:
import calendar
from matplotlib import ticker

ax = plot_data.plot(
    kind='bar', rot=0, xlabel='', ylabel='travelers',
    figsize=(8, 1.5), title='TSA Monthly Traveler Throughput'
)
# abgekürzte Monatsnamen für die x-Achse, aus der stdlib
ax.set_xticklabels(calendar.month_abbr[1:])

# y-Achsenlabels in Millionen statt wissenschaftlicher Notation
ax.yaxis.set_major_formatter(ticker.EngFormatter())

# Legende anpassen und verschieben
ax.legend(title='', loc='center', bbox_to_anchor=(0.5, -0.3), ncols=3, frameon=False)


Anmerkungen:
- Matplotlibs `ticker` Modul hat Funktionalität um die Tick Labels und Platzierungen anzupassen, Dokumentation  [hier](https://matplotlib.org/stable/api/ticker_api.html).
- `plot()` nimmt viele [Parameter](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html) von denen die meisten weiter an Matplotlib gehen. Manchmal müssen wir für Anpassungen aber Matplotlib direkt nutzen.

### Visualisierung von Verteilungen

Wir wollen die Verteilung der täglichen Fluggastwerten darstellen. Dafür bauen wir für jedes Jahr einen Subplot mit Histogram und Kernel Density Estimate (KDE).

Anders als bisher erzeugen wir hier selber ein `Figure` und drei `Axes` Objekte für unser custom Layout. Wir könnten auch in `plot()` angeben dass wir Subplots wollen (mit `subplots` und `layout`), aber der direkte Weg mit Matplotlib gibt uns noch mehr Flexibilität.


In [ ]:
import matplotlib.pyplot as plt
# Erzeuge drei Subplots in einer Spalte, mit den gleichen Achsen.
fig, axes = plt.subplots(3, 1, sharex=True, sharey=True, figsize=(6, 4))

# Iteration über jedes Jahr im DataFrame und jede Suplot-Achse.
for year, ax in zip(tsa_melted_holiday_travel['year'].unique(), axes):
    # Daten auf das Jahr filtern.
    data_year = tsa_melted_holiday_travel[tsa_melted_holiday_travel['year'] == year]
    # Nur die travelers Spalte plotten.
    data_year['travelers'].plot(kind='hist', legend=False, density=True, alpha=0.8, ax=ax)
    data_year['travelers'].plot(kind='kde', legend=False, color='blue', ax=ax)
    ax.set(title=f'{year} TSA Traveler Throughput', xlabel='travelers')
    ax.xaxis.set_major_formatter(ticker.EngFormatter())
    ax.set_xlim(left=0)

fig.tight_layout()

### Übung

##### Mit den Daten in `tsa_melted_holiday_travel.csv`, erzeuge Boxplots für die jährlichen Fluggastzahlen.

Hinweis: 
- wir brauchen ein pivot mit `column='year'` und `values='travelers'`
- nutze `kind='box'` in der `plot()` Methode

In [ ]:
import pandas as pd

df = pd.read_csv('../data/tsa_melted_holiday_travel.csv')

# Lösung hier...

.

.

.

.

.

.

.

.

.

.

### Lösung

In [ ]:
from matplotlib import ticker
import pandas as pd

df = pd.read_csv('../data/tsa_melted_holiday_travel.csv')
plot_data = df.pivot(columns='year', values='travelers')

ax = plot_data.plot(kind='box')
ax.set(xlabel='year', ylabel='travelers', title='TSA Traveler Throughput')
ax.yaxis.set_major_formatter(ticker.EngFormatter())

## Lernpfad

1. Warum ist Visualisierung wichtig?
1. Plotting mit Pandas und Matplotlib
1. **Plotting mit Seaborn**
1. Anpassen von Matplotlib-Plots
1. Interaktive Visualisierung mit Plotly

## Plotting mit Seaborn

Seaborn macht es einfacher, Daten im langen Format direkt, ohne Pivot, zu visualisieren. 

Außerdem bietet es einige zusätzliche Diagrammtypen und hübschere Defaults. 

Seaborn baut auf Matplotlib auf.

### Visualisieren von Daten im langen Format

Wir reproduzieren zuerst das Balkendiagramm von oben:

In [ ]:
import seaborn as sns

plot_data = tsa_melted_holiday_travel.assign(month=tsa_melted_holiday_travel.index.month)
plot_data.head(3)  # noch in langer Form!

In [ ]:
# Pivot wird jetzt direkt in der Plotfunktion durchgeführt.
sns.barplot(
    plot_data, 
    x='month',
    y='travelers', 
    hue='year', 
    estimator='sum', 
    errorbar=None, 
    palette="pastel" 
)

Sowie die gleiche Verteilungsvisualisierung wie oben:

In [ ]:
sns.displot(
    data=tsa_melted_holiday_travel, x='travelers', row='year', kde=True, aspect=4, height=1.5
)

### Heatmaps

Wenn wir Pivot Tables haben, können wir diese mit Seaborn auch als Heatmaps darstellen:

In [ ]:
data = tsa_melted_holiday_travel\
    .assign(month=tsa_melted_holiday_travel.index.month)\
    .pivot_table(index='month', columns='year', values='travelers', aggfunc='sum')
data

In [ ]:
ax = sns.heatmap(data=data / 1e6, cmap='Blues', annot=True, fmt='.1f')
ax.set_yticklabels(calendar.month_abbr[1:], rotation=0)
ax.set_title('Total TSA Traveler Throughput (in millions)')

*Tip: Die Matplotlib Dokumentation hat mehr Informationen über [colormaps](https://matplotlib.org/stable/tutorials/colors/colormaps.html) und [named colors](https://matplotlib.org/stable/gallery/color/named_colors.html).*

Weitere Interssante Beispiele sind:
- [pairwise plots](https://seaborn.pydata.org/generated/seaborn.pairplot.html#seaborn.pairplot) mit `pairplot()`
- [categorical scatter plots](https://seaborn.pydata.org/generated/seaborn.swarmplot.html#seaborn.swarmplot) mit `swarmplot()`
- [joint distribution plots](https://seaborn.pydata.org/generated/seaborn.jointplot.html#seaborn.jointplot) mit `jointplot()`
- [FacetGrids](https://seaborn.pydata.org/generated/seaborn.FacetGrid.html#seaborn.FacetGrid) für custom Layouts mit beliebigen Diagrammtypen

### Übung

Mit den Daten in  `tsa_melted_holiday_travel.csv`, erzeuge eine Heatmap, die den Median der Reisendenzahlen für jeden Wochentag und Monat darstellt.

Vervollständige diesen Anfang:

In [ ]:
import calendar

from matplotlib import ticker
import pandas as pd
import seaborn as sns

def get_data():
    return pd.read_csv(
        '../data/tsa_melted_holiday_travel.csv',
        parse_dates=True, index_col='date'
    )

def heatmap(data):
    data_2019 = data.assign(
        day_of_week=data.index.dayofweek, month=data.index.month
    ).loc['2019']
    
    
    pivot_data = data_2019.pivot_table(
        ...
    )

    ax = sns.heatmap(...)
    # Bonus:
    # ax.set_xticklabels(...)
    # ax.set_yticklabels(...)
    # ax.set_title(...)
    return ax
    

df = get_data()
# heatmap(df)


.

.

.


.



.


.

.

.

.

.


### Lösung

In [ ]:
import calendar

from matplotlib import ticker
import pandas as pd
import seaborn as sns

def get_data():
    return pd.read_csv(
        '../data/tsa_melted_holiday_travel.csv',
        parse_dates=True, index_col='date'
    )

def heatmap(data):
    data_2019 = data.assign(
        day_of_week=data.index.dayofweek, month=data.index.month
    ).loc['2019']
    
    
    pivot_data = data_2019.pivot_table(
        columns='day_of_week', index='month', values='travelers', aggfunc='median'
    )

    ax = sns.heatmap(data=pivot_data / 1e6, annot=True, fmt='.1f', cmap='Blues')
    ax.set_xticklabels(calendar.day_abbr)
    ax.set_yticklabels(calendar.month_abbr[1:], rotation=0)
    ax.set_title('2019 TSA Median Traveler Throughput\n(in millions)')
    return ax
    

df = get_data()
heatmap(df)


## Lernpfad

1. Warum ist Visualisierung wichtig?
1. Plotting mit Pandas und Matplotlib
1. Plotting mit Seaborn
1. **Anpassen von Matplotlib-Plots**
1. Interaktive Visualisierung mit Plotly

## Anpassen von Matplotlib-Plots

Matplotlib ist fast unendlich anpassbar. 

Als Beispiel werden wir unseren Plots hier schattierte Regionen und Anmerkungen hinzufügen.

### Regionen schattieren

Beim Plot der Fluggastzahlen über die Zeit wäre es interessant, die Feiertagszeiten zu sehen. Diese markieren wir mit der `axvspan()` Methode:

In [ ]:
plot_data = tsa_melted_holiday_travel['2019-05':'2019-11']
ax = plot_data.travelers.plot(
    title='TSA Traveler Throughput', ylabel='travelers', figsize=(9, 2)
)
ax.yaxis.set_major_formatter(ticker.EngFormatter())

# Dataframe mit Start und Endzeit der Feiertagsperioden
holiday_ranges = plot_data.dropna().reset_index()\
    .groupby('holiday').agg({'date': ['min', 'max']})

# fügt eine graue Region für jede Periode hinzu
for start_date, end_date in holiday_ranges.itertuples(index=False):
    ax.axvspan(start_date, end_date, color='gray', alpha=0.2)
    

*Tip: `axhspan()` erzeugt horizontale Flächen, `axvline()` / `axhline()` vertikale bzw. horizontale Linien.*

### Anmerkungen

Mit `annotate()` können wir Anmerkungen auf dem Plot hinzufügen. Hier zeigen wir den Tag, der 2019 die höchsten Fluggastzahlen hatte (der Tag nach Thanksgiving).

In [ ]:
plot_data = tsa_melted_holiday_travel.loc['2019']
ax = plot_data.travelers.plot(
    title='TSA Traveler Throughput', ylabel='travelers', figsize=(9, 2)
)
ax.yaxis.set_major_formatter(ticker.EngFormatter())

# Tag mit höchsten Zahlen
max_throughput_date = plot_data.travelers.idxmax()
max_throughput = plot_data.travelers.max()

_ = ax.annotate(
    f'{max_throughput_date:%b %d}\n({max_throughput / 1e6:.2f} M)',
    xy=(max_throughput_date, max_throughput),
    xytext=(max_throughput_date - pd.Timedelta(days=25), max_throughput * 0.92),
    arrowprops={'arrowstyle': '->'}, ha='center'
)

*Anmerkung: Erklärung der Syntax im f-String [hier](https://fstring.help/) und [hier](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes).*

### Was kann man an einem Matplotlib Diagramm alles noch verändern?

Graphische Übersicht der Einstellmöglichkeiten:

![Anatomy of a figure](https://matplotlib.org/stable/_images/sphx_glr_anatomy_001.png)

Quelle und Details [hier](https://matplotlib.org/stable/gallery/showcase/anatomy.html).


## Übung

1. Gehe zur [Matplotlib Gallerie](https://matplotlib.org/stable/gallery/index.html) und kopiere mindestens ein Diagramm.

2. Kopiere ein Diagramm aus der [Seaborn Gallerie](https://seaborn.pydata.org/examples/).